In [17]:
#Justin Garzione
#Copyright MIT License
#Decision Tree -> Continuous Variables

In [18]:
import sys
import math

In [19]:
#DEBUGGING MODULE

#Input 21
f = open("/Users/justingarzione/Documents/Data Mining/Exam 3/tutorials-master/dt_debug_testcases/input21.txt", "r")
f1 = open("/Users/justingarzione/Documents/Data Mining/Exam 3/tutorials-master/dt_debug_testcases/output21.txt", "r")

#Input 22
g = open("/Users/justingarzione/Documents/Data Mining/Exam 3/tutorials-master/dt_debug_testcases/input22.txt", "r")
g1 = open("/Users/justingarzione/Documents/Data Mining/Exam 3/tutorials-master/dt_debug_testcases/output22.txt", "r")

#Input 23
h = open("/Users/justingarzione/Documents/Data Mining/Exam 3/tutorials-master/dt_debug_testcases/input23.txt", "r")
h1 = open("/Users/justingarzione/Documents/Data Mining/Exam 3/tutorials-master/dt_debug_testcases/output23.txt", "r")

#Input 24
i = open("/Users/justingarzione/Documents/Data Mining/Exam 3/tutorials-master/dt_debug_testcases/input24.txt", "r")
i1 = open("/Users/justingarzione/Documents/Data Mining/Exam 3/tutorials-master/dt_debug_testcases/output24.txt", "r")

In [20]:
%%time
def read_ans(x):
    Lines = x.readlines()
    out = []
    for line in Lines:
        out.append(int(line.strip()))
    return out

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 10 µs


In [21]:
%%time
def read_data(data):
    attribute_list = []
    attribute_list.append("Labels")
    attribute_index = 0
    training_data = []
    testing_data = []
    for current_line in data:
        temp = current_line.strip('\n').split(' ')
        line_list = []
        line_list.append(int(temp[0]))
        for element in range(1,len(temp)):
            if ":" in temp[element]:
                colon_index = temp[element].index(":")
                
                if attribute_index < len(temp)-1:
                    attribute = int(temp[element][0:colon_index])
                    attribute_list.append(attribute)
                    attribute_index+=1
                
                value = float(temp[element][colon_index+1:])
                line_list.append(value)
                
        if line_list[0] == -1:
            testing_data.append(tuple(line_list))
            
        else:
            training_data.append(tuple(line_list))
    
    attribute_data = tuple(attribute_list)
    return training_data, testing_data, attribute_data

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11.2 µs


In [22]:
%%time
def class_counts(rows):
    counts = {}
    highest_key = 0
    for row in rows:
        label = row[0]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    for element in counts.keys():
        if counts[element] > highest_key:
            highest_key = element
    
    return counts,highest_key

CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 15.3 µs


In [23]:
%%time
def get_info(data,column=0):
    denom = len(data)
    info_dict = dict()
    info = 0
    for row in data:
        if row[0] not in info_dict.keys():
            info_dict[row[column]] = 0
        info_dict[row[column]] += 1
        
    for val in info_dict.values():
        info += -(val/denom) * math.log(val/denom,2)
    return info

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [24]:
%%time
def attribute_info(split_points,training_data,column,attribute_data,dataset_info):
    best_gain = 0
    best_split_point = 0
    lowest_attribute = 0
    best_less_than = []
    best_more_than = []
    for point in split_points:
        less_than = []
        more_than = []
        for row in training_data:
            if row[column] <= point:
                less_than.append(row)
            else:
                more_than.append(row)
        
        weight_left = len(less_than)
        info_left = get_info(less_than)
        weight_right = len(more_than)
        info_right = get_info(more_than)
        denom = weight_left + weight_right

        split_info = ((weight_left/denom)*info_left) + ((weight_right/denom)*info_right)
        gain = dataset_info - split_info 
        
        if gain > best_gain or (gain == best_gain and attribute_data[column] < lowest_attribute):
                                best_gain = gain
                                lowest_attribute = attribute_data[column]
                                best_less_than = less_than
                                best_more_than = more_than
                                best_split_point = point
    return best_gain,best_less_than,best_more_than,lowest_attribute,best_split_point

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.87 µs


In [25]:
%%time
def find_best_split(training_data):
    dataset_info_ = get_info(training_data)
    split_points = []
    best_gain = 0
    best_point = 0
    best_true_rows = []
    best_false_rows = []
    lowest_attribute = 0
    best_question = Question(0,0)
    #Sort the attribute/value in increasing order
    for column in range(1,len(training_data[0])):
        training_data.sort(key=lambda pair: pair[column])

        #Get a list of possible split points
        for row in range(0,len(training_data)-1):
            col_midpoint = ((training_data[row][column] + training_data[row+1][column]) / 2)
            split_points.append(col_midpoint)

        #Call Information Gain Function
        gain,less_than,more_than,low_attribute,point  = attribute_info(split_points,training_data,column,attribute_data,dataset_info_)
        if gain > best_gain or (gain == best_gain and low_attribute < lowest_attribute):
            best_gain = gain
            best_true_rows = less_than
            best_false_rows = more_than
            lowest_attribute = low_attribute
            best_point = point
            best_question = Question(column,best_point)

        split_points = []

    return best_gain,best_question,best_true_rows,best_false_rows 

CPU times: user 7 µs, sys: 9 µs, total: 16 µs
Wall time: 17.9 µs


In [26]:
%%time
class Question:
    
    def __init__(self,column,value):
        self.column = column
        self.value = value
        
    def match(self, example):
        val = example[self.column]
        return val <= self.value
    

CPU times: user 21 µs, sys: 0 ns, total: 21 µs
Wall time: 23.8 µs


In [27]:
%%time
class Leaf:
    def __init__(self,rows):
        self.predictions, self.win = class_counts(rows)

CPU times: user 30 µs, sys: 1 µs, total: 31 µs
Wall time: 35 µs


In [28]:
%%time
class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

CPU times: user 32 µs, sys: 0 ns, total: 32 µs
Wall time: 36 µs


In [29]:
%%time
def build_tree(rows,depth):
    
    gain, question,true_rows,false_rows = find_best_split(rows)
    
    if gain == 0 or depth == 2:
        return Leaf(rows)
    
    true_branch = build_tree(true_rows,depth+1)
    false_branch = build_tree(false_rows,depth+1)
    
    return Decision_Node(question,true_branch,false_branch)

CPU times: user 6 µs, sys: 13 µs, total: 19 µs
Wall time: 21.9 µs


In [30]:
%%time
def classify(row,node):
    
    if isinstance(node,Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)

CPU times: user 13 µs, sys: 1 µs, total: 14 µs
Wall time: 18.8 µs


In [31]:
%%time
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    highest_prob = 0
    highest_key = 0
    for lbl in counts.keys():
        probs[lbl] = float(int(counts[lbl] / total * 100))
        if probs[lbl] > highest_prob or (probs[lbl] == highest_prob and lbl < highest_key):
            highest_prob = probs[lbl]
            highest_key = lbl

    return highest_key

CPU times: user 14 µs, sys: 11 µs, total: 25 µs
Wall time: 148 µs


In [32]:
%%time
training_data, testing_data, attribute_data = read_data(i)
my_tree = build_tree(training_data,depth=0)

my_ans = []
for row in testing_data:
           my_ans.append(print_leaf(classify(row, my_tree)))

correct = read_ans(i1)
print(f"Model: {my_ans}")
print(f"Correct: {correct}")
print(np.array(my_ans) == np.array(correct))


Model: [3, 1, 3, 3, 1, 1, 1, 1, 4, 1, 3, 4, 3, 1, 4, 1, 3, 3, 3, 4]
Correct: [3, 1, 3, 3, 1, 1, 1, 1, 4, 1, 3, 4, 3, 1, 4, 1, 3, 3, 3, 4]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
CPU times: user 16.4 ms, sys: 3.07 ms, total: 19.5 ms
Wall time: 19.5 ms
